In [15]:
!pip install accelerate -U
!pip install pandas
!pip install konlpy
!pip install transformers
!pip install torch

In [16]:
!pip install konlpy
!git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git
%cd Mecab-ko-for-Google-Colab/
!bash install_mecab-ko_on_colab_light_220429.sh
%cd ../

fatal: destination path 'Mecab-ko-for-Google-Colab' already exists and is not an empty directory.
/home/kannakawai9/sessac_project/Mecab-ko-for-Google-Colab
install_mecab-ko_on_colab_light_220429.sh: line 4: cd: /content: No such file or directory
Installing konlpy.....
Done
Installing mecab-0.996-ko-0.9.2.tar.gz.....
from https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
--2024-03-27 01:12:49--  https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
Resolving bitbucket.org (bitbucket.org)... 104.192.141.1, 2406:da00:ff00::22c2:513, 2406:da00:ff00::22cd:e0db, ...
Connecting to bitbucket.org (bitbucket.org)|104.192.141.1|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://bbuseruploads.s3.amazonaws.com/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz?response-content-disposition=attachment%3B%20filename%3D%22mecab-0.996-ko-0.9.2.tar.gz%22&response-content-encoding=None&AWSAccessKeyId=ASIA6KOSE3BNME

In [17]:
!pip install wandb
import wandb
wandb.login()

True

In [18]:
import os
os.environ["WANDB_PROJECT"] = "classification"

In [19]:
!pip install scikit-learn

In [20]:
import pandas as pd
from konlpy.tag import Mecab
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
import torch
import numpy as np
import re

In [21]:
df = pd.read_csv('/home/kannakawai9/sessac_project/data_processed.csv')

In [22]:
from sklearn.model_selection import train_test_split

# 'text'와 'label' 열만 사용하려면
df = df[['text', 'label']]

# 데이터프레임을 훈련 세트와 테스트 세트로 분할
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)


In [23]:
# 레이블을 정수로 매핑하는 딕셔너리를 만듭니다.
label_map = {'정치': 0, '경제': 1, '사회': 2, '국제': 3}

# 이 딕셔너리를 데이터프레임에 적용합니다.
train_df['label'] = train_df['label'].map(label_map)
test_df['label'] = test_df['label'].map(label_map)


In [24]:
!pip install datasets


# VALIDATION 추가

In [25]:
df = pd.read_csv('/home/kannakawai9/sessac_project/data_processed.csv')

label_map = {'정치': 0, '경제': 1, '사회': 2, '국제': 3}

# 이 딕셔너리를 데이터프레임에 적용합니다.
df['label'] = df['label'].map(label_map)

In [ ]:
# 필요한 라이브러리를 설치합니다.
import torch
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset
from konlpy.tag import Mecab
import wandb

# wandb를 초기화합니다.
wandb.init(project="text-classification")

# GPU 사용 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 모델과 토크나이저를 불러옵니다.
model_name = "bert-base-multilingual-cased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=4).to(device)

# Mecab 객체를 생성합니다.
mecab = Mecab()

def tokenize(batch):
    # 텍스트를 Mecab으로 토크나이징합니다.
    tokenized_texts = [' '.join(mecab.morphs(text)) for text in batch['text']]
    # 토크나이징된 텍스트를 Hugging Face의 토크나이저에 전달합니다.
    encoding = tokenizer(
        tokenized_texts,
        padding='max_length',
        truncation=True,
        max_length=512,
        return_tensors='pt'
    )
    return encoding

# 데이터프레임을 Hugging Face의 Dataset으로 변환
df = Dataset.from_pandas(df[['text', 'label']])

# Dataset을 pandas 데이터프레임으로 변환합니다.
df = df.to_pandas()

# 데이터프레임을 훈련 세트, 검증 세트, 테스트 세트로 분할
train_df, temp_df = train_test_split(df, test_size=0.3, random_state=42)
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)

# 이제 map 함수를 사용할 수 있습니다.
train_dataset = Dataset.from_pandas(train_df).map(tokenize, batched=True)
val_dataset = Dataset.from_pandas(val_df).map(tokenize, batched=True)
test_dataset = Dataset.from_pandas(test_df).map(tokenize, batched=True)

# 데이터셋을 PyTorch DataLoader 형식으로 변환합니다.
train_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])
val_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])
test_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])

# 훈련 설정
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,  # 또는 원하는 에포크 수로 설정
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    warmup_steps=140,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    evaluation_strategy="steps",  # 검증 세트를 사용하여 과적합을 모니터링합니다.
    eval_steps=10,  # 매 10번의 학습 스텝마다 검증을 수행합니다.
    report_to="wandb"  # Weights & Biases에 로그를 보냅니다.
)

# 훈련 시작
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset  # 검증 세트를 지정합니다.
)

trainer.train()


In [32]:
# 체크포인트를 저장할 파일 이름을 정의합니다.
checkpoint_path = "/home/kannakawai9/sessac_project/epochs_3_t_v_t.pt"

# 모델의 state_dict를 저장합니다.
torch.save(model.state_dict(), checkpoint_path)


In [25]:
# 예측 결과와 레이블 비교

predictions = trainer.predict(test_dataset)

_, predicted = torch.max(torch.tensor(predictions.predictions), 1)
labels = torch.tensor(predictions.label_ids)

# 정확도 계산
correct_predictions = (predicted == labels).sum().item()
total_samples = len(labels)
accuracy = correct_predictions / total_samples

print(f'Accuracy: {accuracy * 100:.2f}%')


Accuracy: 90.28%


In [26]:

# 카테고리 목록을 정의합니다.
categories = ['정치', '경제', '사회', '국제']

def predict(text):
    # 텍스트를 Mecab으로 토크나이징합니다.
    tokens = mecab.morphs(text)

    # 토크나이징된 텍스트를 Hugging Face의 토크나이저에 전달합니다.
    inputs = tokenizer(' '.join(tokens), padding=True, truncation=True, max_length=512, return_tensors='pt').to(device)

    # 모델을 평가 모드로 설정합니다.
    model.eval()

    # 예측을 수행합니다.
    with torch.no_grad():
        outputs = model(**inputs)

    # 출력에서 가장 높은 값을 가진 클래스를 찾습니다.
    _, predicted = torch.max(outputs.logits, 1)

    # 예측된 클래스를 해당하는 카테고리 이름으로 변환합니다.
    category = categories[predicted.item()]

    return category


In [27]:
predict(input())

'사회'